In [ ]:
# default_exp cleaner

# Cleaner

> The clearners are the last but not least blocks of web2dataset. Their goal is to purge and clean the dataset.

Example of cleaner (not yet implemented):

* delete double (based on hash)
* delete image with low resolution
* ethic base purger, how to ?



In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from abc import ABC, abstractmethod

# ImageCleaner

In [ ]:
# export
class ImageCleaner:
    def __init__(self, path: str):
        self.path = path

    def clean(self):
        pass

In [ ]:
class DuplicateImageCleaner(ImageCleaner):
    pass